In [ ]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import IPython
import ipywidgets as iw
from stimuli.utils import plot_stimuli, plot_stim

# narrowband

In [ ]:
from stimuli.noises import narrowband

## Parameterization

In [ ]:
params = {
    "visual_size": (10, 10),
    "ppd": 40.,
    "center_frequency": 3,
    "bandwidth": 1,
    "rms_contrast": 0.2,
    "pseudo_noise": True,
}

stim = narrowband(**params)
plot_stim(stim)
plt.show()

## Interactive

In [ ]:
# Define widgets
w_height = iw.IntSlider(value=15, min=5, max=25, description="height [deg]")
w_width = iw.IntSlider(value=15, min=5, max=25, description="width [deg]")
w_ppd = iw.IntSlider(value=40, min=20, max=60, step=2, description="ppd")
w_size = iw.HBox([w_height, w_width, w_ppd])

w_freq = iw.FloatSlider(value=3., min=0.1, max=10.0, description="center frequency")
w_band = iw.FloatSlider(value=1., min=0.5, max=2.0, description="bandwidth (octaves)")
w_stats = iw.HBox([w_freq, w_band])

w_rms = iw.FloatSlider(value=0.5, min=0.1, max=1.0, description="rms contrast (std)")
ui = iw.VBox([w_size, w_stats, w_rms])

def show_noise(
    height=None,
    width=None,
    ppd=None,
    center_frequency=None,
    bandwidth=None,
    rms_contrast=None,
):

    stim = narrowband(
        visual_size=(height, width),
        ppd=ppd,
        center_frequency=center_frequency,
        bandwidth=bandwidth,
        rms_contrast=rms_contrast
    )
    plot_stim(stim)


out = iw.interactive_output(show_noise,
                            {
                                "height": w_height,
                                "width": w_width,
                                "ppd": w_ppd,
                                "center_frequency": w_freq,
                                "bandwidth": w_band,
                                "rms_contrast": w_rms,
                            })

display(ui, out)


## Center frequency and bandwidth

In [ ]:
def show_noise_properties(stim):
    # Get amplitude spectrum
    noise_amplitude = np.abs(np.fft.fftshift(np.fft.fft2(stim["img"])))
    fs = np.fft.fftshift(np.fft.fftfreq(stim["shape"][0], d=1./stim["ppd"]))
    
    # Calculate full-width-half-maximum
    fwhm = stim["sigma"] * 2. * np.sqrt(2. * np.log(2))
    
    # Get lower and upper bound at fwhm
    fup, flow = stim["center_frequency"] + fwhm/2., stim["center_frequency"] - fwhm/2.
    
    # Calculate ratio and fractional bandwidth
    ratio_bandwidth = np.log2(fup / flow)  # in octaves
    frac_bandwidth = (fup-flow) / stim["center_frequency"]

    # Plotting:
    plt.figure(figsize=(18, 4))
    plt.subplot(1, 2, 1), plt.imshow(noise_amplitude, extent=[fs[0], fs[-1],]*2)
    plt.colorbar(), plt.title('Noise spectrum'), plt.xlabel('cpd')

    plt.subplot(1, 2, 2)
    plt.plot(fs, noise_amplitude[int(noise_amplitude.shape[0]/2.), :])
    plt.plot([flow, flow], [0., noise_amplitude.max()], label='lower boundary max/2')
    plt.plot([fup, fup], [0., noise_amplitude.max()], label='upper boundary max/2')
    plt.title('Cut through'), plt.xlabel('cpd'), plt.legend()
    plt.show()

    print('Center frequency:', np.round(stim["center_frequency"], 3))
    print('Ratio bandwidth:', np.round(ratio_bandwidth, 3))
    print('Fractional bandwidth:', np.round(frac_bandwidth, 3))
    return

In [ ]:
def interact_noise_properties(center_frequency, bandwidth):
    stim = narrowband(visual_size=(10, 10), ppd=40, center_frequency=center_frequency, bandwidth=bandwidth)
    show_noise_properties(stim)

iw.interact(interact_noise_properties, center_frequency=(0.5, 9.5), bandwidth=(0.5, 1.5))

## Random noise vs pseudo-random noise

In [ ]:
def interact_randomness(center_frequency, bandwidth):
    stim1 = narrowband(visual_size=(10, 10), ppd=40, center_frequency=center_frequency, pseudo_noise=True)
    stim2 = narrowband(visual_size=(10, 10), ppd=40, center_frequency=center_frequency, pseudo_noise=False)
    show_noise_properties(stim1)
    show_noise_properties(stim2)

iw.interact(interact_randomness, center_frequency=(0.5, 9.5), bandwidth=(0.5, 1.5))